In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
base_path = '../../'
sys.path.append(base_path)

In [5]:
from src.dgl_flow_field_dataset import DGLSurfaceFlowFieldDataset
from src.pyvista_flow_field_dataset import PyvistaFlowFieldDataset
ds_pv = PyvistaFlowFieldDataset.load_from_huggingface(num_samples=3)
ds_dgl = DGLSurfaceFlowFieldDataset(os.path.join(base_path,'datasets/dgl_surface'),ds_pv)
#ds_dgl = DGLSurfaceFlowFieldDataset(os.path.join(base_path,'datasets/dgl_surface'))

Found 333 files in repository 'peteole/CoolMucSmall'.


volume_design_1.cgns:   0%|          | 0.00/44.8M [00:00<?, ?B/s]

volume_design_10.cgns:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

volume_design_1001.cgns:   0%|          | 0.00/46.3M [00:00<?, ?B/s]

surface_design_1.cgns:   0%|          | 0.00/3.24M [00:00<?, ?B/s]

surface_design_10.cgns:   0%|          | 0.00/3.17M [00:00<?, ?B/s]

surface_design_1001.cgns:   0%|          | 0.00/3.34M [00:00<?, ?B/s]

All files have been downloaded to 'datasets/ds_huggingface'.


In [18]:
ds_pv[0].get_surface_points(block_index=0)

pyvista_ndarray([[ 0.4       , -0.0655799 ,  0.02      ],
                 [ 0.398     , -0.0635799 ,  0.02      ],
                 [ 0.4       , -0.0615799 ,  0.02      ],
                 ...,
                 [ 0.184     , -0.0355799 ,  0.02      ],
                 [ 0.16574037, -0.03541487,  0.        ],
                 [ 0.18      , -0.0395799 ,  0.02      ]],
                shape=(17957, 3))

In [22]:
ds_pv[1].plot_volume("Pressure")

Widget(value='<iframe src="http://localhost:65163/index.html?ui=P_0x34898f950_9&reconnect=auto" class="pyvista…

In [6]:
ds_dgl[0]

Graph(num_nodes=21428, num_edges=84248,
      ndata_schemes={'BodyID': Scheme(shape=(), dtype=torch.int32), 'CellArea': Scheme(shape=(), dtype=torch.float32), 'Normal': Scheme(shape=(3,), dtype=torch.float32), 'ShearStress': Scheme(shape=(3,), dtype=torch.float32), 'Position': Scheme(shape=(3,), dtype=torch.float32), 'Temperature': Scheme(shape=(), dtype=torch.float32), 'Pressure': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={'dx': Scheme(shape=(3,), dtype=torch.float32)})

In [7]:
from dgl.dataloading import GraphDataLoader
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataloader = GraphDataLoader(ds_dgl, batch_size=1, shuffle=True)
for batch in dataloader:
    print(batch)
    

Graph(num_nodes=22474, num_edges=88136,
      ndata_schemes={'BodyID': Scheme(shape=(), dtype=torch.int32), 'CellArea': Scheme(shape=(), dtype=torch.float32), 'Normal': Scheme(shape=(3,), dtype=torch.float32), 'ShearStress': Scheme(shape=(3,), dtype=torch.float32), 'Position': Scheme(shape=(3,), dtype=torch.float32), 'Temperature': Scheme(shape=(), dtype=torch.float32), 'Pressure': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={'dx': Scheme(shape=(3,), dtype=torch.float32)})
Graph(num_nodes=21460, num_edges=84374,
      ndata_schemes={'BodyID': Scheme(shape=(), dtype=torch.int32), 'CellArea': Scheme(shape=(), dtype=torch.float32), 'Normal': Scheme(shape=(3,), dtype=torch.float32), 'ShearStress': Scheme(shape=(3,), dtype=torch.float32), 'Position': Scheme(shape=(3,), dtype=torch.float32), 'Temperature': Scheme(shape=(), dtype=torch.float32), 'Pressure': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={'dx': Scheme(shape=(3,), dtype=torch.float32)})
Graph(num_node

In [8]:
import dgl


def get_node_edge_X(graph: dgl.DGLGraph):
    node_X = torch.cat([graph.ndata["Position"], graph.ndata["Normal"]],dim=1)
    edge_X = torch.cat([graph.edata["dx"]],dim=1)
    return node_X, edge_X

def get_node_Y(graph: dgl.DGLGraph):
    return torch.cat([graph.ndata["Pressure"].unsqueeze(1),graph.ndata["Temperature"].unsqueeze(1),graph.ndata['ShearStress']],dim=1)

def set_graph_features(graph: dgl.DGLGraph, node_X, edge_X, node_Y):
    graph.ndata["Position"] = node_X[:,:3]
    graph.ndata["Normal"] = node_X[:,3:]
    graph.edata["dx"] = edge_X
    graph.ndata["Pressure"] = node_Y[:,0]
    graph.ndata["Temperature"] = node_Y[:,1]
    graph.ndata["ShearStress"] = node_Y[:,2:]
g=ds_dgl[0]
g_cp=g.clone()
ndx, edx = get_node_edge_X(g)
ndy = get_node_Y(g)
set_graph_features(g_cp, ndx, edx, ndy)
assert torch.allclose(g_cp.ndata["Position"], g.ndata["Position"])
assert torch.allclose(g_cp.ndata["Normal"], g.ndata["Normal"])
assert torch.allclose(g_cp.edata["dx"], g.edata["dx"])
assert torch.allclose(g_cp.ndata["Pressure"], g.ndata["Pressure"])
assert torch.allclose(g_cp.ndata["Temperature"], g.ndata["Temperature"])
assert torch.allclose(g_cp.ndata["ShearStress"], g.ndata["ShearStress"])
num_node_features = ndx.shape[1]
num_edge_features = edx.shape[1]
num_node_labels = ndy.shape[1]
print("Node X: ",ndx, ndx.shape)
print("Edge X: ",edx, edx.shape)
print("Node Y: ",ndy, ndy.shape)

Node X:  tensor([[-1.7375e+00,  1.7359e+00, -1.1969e+00, -7.7721e-03, -1.5141e-02,
         -1.0316e+00],
        [-1.7201e+00,  1.7360e+00, -1.1969e+00, -7.7721e-03, -1.5141e-02,
         -1.0316e+00],
        [-1.7374e+00,  1.6648e+00, -1.1969e+00, -7.7721e-03, -1.5141e-02,
         -1.0316e+00],
        ...,
        [ 1.6292e+00, -3.5846e-03, -1.3716e-01, -6.3510e+00, -1.5141e-02,
          7.7114e-03],
        [ 1.6228e+00, -3.5846e-03, -9.1087e-02, -7.7721e-03, -1.5141e-02,
         -1.0316e+00],
        [ 1.6228e+00, -2.2674e-02, -1.3716e-01, -7.7721e-03,  4.4902e+00,
          7.7114e-03]]) torch.Size([21428, 6])
Edge X:  tensor([[ 1.4280e+00,  1.7693e-04,  2.7578e-10],
        [ 6.6880e-03, -1.4613e+00,  2.7578e-10],
        [-1.4280e+00, -1.7693e-04,  2.7578e-10],
        ...,
        [ 5.2391e-01,  3.9186e-01,  2.7578e-10],
        [-9.1376e-10,  3.9186e-01,  1.2818e+00],
        [-9.1376e-10,  1.3555e-09, -4.3580e+00]]) torch.Size([84248, 3])
Node Y:  tensor([[ 2.2599, -0.53

In [9]:
from modulus.models.meshgraphnet import MeshGraphNet
model = MeshGraphNet(
    input_dim_nodes=num_node_features,
    input_dim_edges=num_edge_features,
    output_dim=num_node_labels,
    aggregation='sum',
    hidden_dim_edge_encoder=64,
    hidden_dim_node_encoder=64,
    hidden_dim_processor=64,
    hidden_dim_node_decoder=64
)
model=model.to(device)
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
from torch.amp import GradScaler
optimizer = Adam(model.parameters(), lr=1e-4)
scheduler = LambdaLR(optimizer, lr_lambda=lambda epoch: 0.99985 ** epoch)
scaler = GradScaler(device=device)

In [ ]:

from modulus.launch.utils import save_checkpoint, load_checkpoint
checkpoint_path = 'checkpoints'
os.makedirs(checkpoint_path,exist_ok=True)
epoch_init = load_checkpoint(checkpoint_path,model,optimizer,scheduler,scaler,device=device)
num_epochs = 400
model.to(device)
for epoch in range(epoch_init,num_epochs):
    total_loss = 0
    for batch in dataloader:
        batch = batch.to(device)
        node_X, edge_X = get_node_edge_X(batch)
        node_Y = get_node_Y(batch)
        node_Y_pred = model(node_X,edge_X,batch)
        batch_pred_graph = batch.clone()
        set_graph_features(batch_pred_graph, node_X, edge_X, node_Y_pred)
        agg_force_pred = ds_dgl.compute_aggregate_force(batch_pred_graph)
        agg_force = ds_dgl.compute_aggregate_force(batch)
        #print('Agg force pred: ',format_vector(agg_force_pred.tolist()),' Agg force: ',format_vector(agg_force.tolist()))
        loss = torch.nn.functional.mse_loss(node_Y_pred,node_Y) + 5* torch.nn.functional.mse_loss(agg_force_pred,agg_force)
        total_loss += loss.item()
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
    print(f'Epoch {epoch} loss: {total_loss}')
    scheduler.step()
    if epoch % 10 == 0:
        save_checkpoint(checkpoint_path,model,optimizer,scheduler,scaler,epoch)
save_checkpoint(checkpoint_path,model,optimizer,scheduler,scaler,num_epochs)

Epoch 300 loss: 1.3478957563638687
Epoch 301 loss: 1.374155730009079
Epoch 302 loss: 1.3570579588413239
Epoch 303 loss: 1.3635932803153992
Epoch 304 loss: 1.3247084468603134
Epoch 305 loss: 1.356018304824829
Epoch 306 loss: 1.3977064192295074
Epoch 307 loss: 1.392484426498413
Epoch 308 loss: 1.3638753145933151
Epoch 309 loss: 1.3507467359304428
Epoch 310 loss: 1.352421686053276
Epoch 311 loss: 1.3406947553157806
Epoch 312 loss: 1.3524826765060425
Epoch 313 loss: 1.3624065518379211
Epoch 314 loss: 1.3322982341051102
Epoch 315 loss: 1.3381107598543167
Epoch 316 loss: 1.338989645242691
Epoch 317 loss: 1.352041557431221
Epoch 318 loss: 1.3522373288869858
Epoch 319 loss: 1.3334715366363525
Epoch 320 loss: 1.3245503455400467
Epoch 321 loss: 1.352649301290512
Epoch 322 loss: 1.3409426659345627
Epoch 323 loss: 1.3170555979013443
Epoch 324 loss: 1.3337657153606415
Epoch 325 loss: 1.3333241194486618
Epoch 326 loss: 1.3390883803367615
Epoch 327 loss: 1.3376052677631378
Epoch 328 loss: 1.321272104

In [35]:
model.to(device)
g=ds_dgl[0].to(device)
g_pred = g.clone().to(device)
ndx, edx = get_node_edge_X(g)
with torch.no_grad():
    y_pred = model(ndx,edx,g)
set_graph_features(g_pred, ndx, edx, y_pred)
ds_dgl.plot_surface(g_pred,"Pressure")

Widget(value='<iframe src="http://localhost:43085/index.html?ui=P_0x7ef9a6f13c50_18&reconnect=auto" class="pyv…

In [36]:
ds_dgl.plot_surface(g,"Pressure")

Widget(value='<iframe src="http://localhost:43085/index.html?ui=P_0x7ef9a17fa450_19&reconnect=auto" class="pyv…

In [37]:
ds_dgl.compute_aggregate_force(g, 1)

tensor([ 0.0140, -0.0021, -0.0022], device='cuda:0')

In [38]:
ds_dgl.compute_aggregate_force(g_pred,1)

tensor([ 0.0089, -0.0023, -0.0045], device='cuda:0')